In [29]:
import numpy as np
def extract_features(file_name):

    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        audio = librosa.resample(audio, sample_rate, 16000)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=128)
        mfccsscaled = np.mean(mfccs.T,axis=0)

    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None

    return mfccsscaled


# Load various imports
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset
fulldatasetpath = './audio/'

# metadata = pd.read_csv(fulldatasetpath + '../metadata/UrbanSound8K.csv')

features = []

# Iterate through each sound file and extract the features
# for index, row in metadata.iterrows():
#
#     file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#
#     class_label = row["class_name"]
#     data = extract_features(file_name)
#
#     features.append([data, class_label])

for filename in os.listdir('../footstepss'):
    if not filename.endswith('.wav'):
        continue;
    class_label = "footstepss"
    data = extract_features('../footstepss/' + filename)
    features.append([data, class_label])

for filename in os.listdir('../selected'):
    if not filename.endswith('.wav'):
        continue;
    class_label = "selected"
    data = extract_features('../selected/' + filename)
    features.append([data, class_label])


# Convert into a Panda dataframe
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

# split the dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.1, random_state = 420)

Finished feature extraction from  1521  files


In [30]:
print(len(x_train))
x_train.shape

1368


(1368, 128)

In [31]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics

num_rows = 8
num_columns = 16
num_channels = 1

x_train2 = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
x_test2 = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)

num_labels = yy.shape[1]
filter_size = 2

# Construct model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=1, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=1))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=1))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=1))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=1, activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=1))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.GlobalAveragePooling2D())

model.add(tf.keras.layers.Dense(num_labels, activation='softmax'))

In [32]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test2, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy) 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 8, 16, 16)         32        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 8, 16, 16)         0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 8, 16, 16)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 8, 16, 32)         544       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 8, 16, 32)         0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 8, 16, 32)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 8, 16, 64)        

In [33]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 72
num_batch_size = 128

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='saved_models/weights.best.basic_cnn.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train2, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test2, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 1368 samples, validate on 153 samples
Epoch 1/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.7515 - accuracy: 0.5109
Epoch 00001: val_loss improved from inf to 0.71433, saving model to saved_models/weights.best.basic_cnn.hdf5
1368/1368 [==============================] - 1s 1ms/sample - loss: 0.7493 - accuracy: 0.5154 - val_loss: 0.7143 - val_accuracy: 0.5359
Epoch 2/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.7002 - accuracy: 0.5422
Epoch 00002: val_loss improved from 0.71433 to 0.69051, saving model to saved_models/weights.best.basic_cnn.hdf5
1368/1368 [==============================] - 1s 621us/sample - loss: 0.6989 - accuracy: 0.5439 - val_loss: 0.6905 - val_accuracy: 0.4575
Epoch 3/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.6992 - accuracy: 0.4867
Epoch 00003: val_loss improved from 0.69051 to 0.68791, saving model to saved_models/weights.best.basic_cnn.hdf5
1368/1368 [==============================] - 1s 622u

1280/1368 [===========================>..] - ETA: 0s - loss: 0.5830 - accuracy: 0.6773
Epoch 00026: val_loss did not improve from 0.60814
1368/1368 [==============================] - 1s 767us/sample - loss: 0.5844 - accuracy: 0.6769 - val_loss: 0.6319 - val_accuracy: 0.6667
Epoch 27/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5767 - accuracy: 0.6820
Epoch 00027: val_loss did not improve from 0.60814
1368/1368 [==============================] - 1s 618us/sample - loss: 0.5736 - accuracy: 0.6849 - val_loss: 0.6147 - val_accuracy: 0.6536
Epoch 28/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5755 - accuracy: 0.6836
Epoch 00028: val_loss did not improve from 0.60814
1368/1368 [==============================] - 1s 574us/sample - loss: 0.5796 - accuracy: 0.6842 - val_loss: 0.6302 - val_accuracy: 0.6667
Epoch 29/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5792 - accuracy: 0.6898
Epoch 00029: val_loss did not improve from 0.60814
1

1280/1368 [===========================>..] - ETA: 0s - loss: 0.5479 - accuracy: 0.7375
Epoch 00053: val_loss did not improve from 0.57327
1368/1368 [==============================] - 1s 628us/sample - loss: 0.5428 - accuracy: 0.7420 - val_loss: 0.5798 - val_accuracy: 0.6797
Epoch 54/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5417 - accuracy: 0.7437
Epoch 00054: val_loss did not improve from 0.57327
1368/1368 [==============================] - 1s 603us/sample - loss: 0.5362 - accuracy: 0.7471 - val_loss: 0.5750 - val_accuracy: 0.6928
Epoch 55/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5407 - accuracy: 0.7461
Epoch 00055: val_loss did not improve from 0.57327
1368/1368 [==============================] - 1s 608us/sample - loss: 0.5422 - accuracy: 0.7427 - val_loss: 0.5763 - val_accuracy: 0.6928
Epoch 56/72
1280/1368 [===========================>..] - ETA: 0s - loss: 0.5384 - accuracy: 0.7352
Epoch 00056: val_loss did not improve from 0.57327
1

In [34]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train2, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test2, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.73976606
Testing Accuracy:  0.7254902


In [35]:
model.save('./saved_models/model5.h5')


In [36]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()
open("converted_model5.tflite", "wb").write(quantized_model)


17864

NameError: name 'model' is not defined